In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker # 1 

engine = create_engine('sqlite:///:memory:')

Session = sessionmaker(bind=engine) # 2 

session = Session() # 3

In [2]:
from datetime import datetime

from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean, DateTime,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()


class Users(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return "Users(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)

    
class Orders(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)
    
    rel_user=relationship("Users", backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Orders(user_id='{self.user_id}')".format(self=self)
    

class Line_items(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)
    
    rel_order=relationship("Orders", backref=backref('line_items', order_by=item_id))
    rel_listing=relationship("Listings", uselist=False)
    
    def __repr__(self):
        return "Line_items(order_id='{self.order_id}', " \
                        "listing_id='{self.listing_id}', " \
                        "item_start_date='{self.item_start_date}', " \
                        "item_end_date='{self.item_end_date}')".format(self=self)


class Hosts(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)
    def __repr_(self):
        return "Hosts(host_id='{self.host_name}')".format(self=self)
    
    
class Neighbourhoods(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)
    def __repr_(self):
        return "Neighbourhoods(neigh_name='{self.neigh_name}')".format(self=self)
    
    
class Room_types(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)
    def __repr_(self):
        return "Room_types(room_type_name='{self.room_type_name}')".format(self=self)
    

class Property_types(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)
    def __repr_(self):
        return "Property_types(property_type_name='{self.property_type_name}')".format(self=self)

In [3]:
class Listings(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        ForeignKeyConstraint(['host_id'], ['hosts.host_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    rel_neigh = relationship("Neighbourhoods", backref=backref("listings", order_by=listing_id))
    rel_pt = relationship("Property_types", backref=backref("listings", order_by=listing_id))
    rel_rt = relationship("Room_types", backref=backref("listings", order_by=listing_id))
    rel_host = relationship("Hosts", backref=backref("listings", order_by=listing_id))
    rel_li = relationship("Line_items", backref=backref("listings", order_by=listing_id))
    

In [4]:
Base.metadata.create_all(engine)

In [5]:
# data insert
cUser = Users(username="stud", email_address="stud@fa.ru", phone="+79165555555", password="1234")

session.add(cUser)
session.commit()


In [11]:
import pandas as pd
df = pd.read_csv('./ListingsAm.csv', sep=";")
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,Random
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58,0
1,27886,https://www.airbnb.com/rooms/27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,t,Centrum-West,Private room in houseboat,Private room,1.5 baths,...,2020-07-25,99,10.0,10.0,10.0,10.0,10.0,10.0,2.01,0
2,28871,https://www.airbnb.com/rooms/28871,Comfortable double room,124245,Edwin,t,Centrum-Oost,Private room in apartment,Private room,1 shared bath,...,2020-09-20,97,10.0,10.0,10.0,10.0,10.0,10.0,2.68,0
3,29051,https://www.airbnb.com/rooms/29051,Comfortable single room,124245,Edwin,t,Centrum-Oost,Private room in apartment,Private room,1 shared bath,...,2020-08-28,95,10.0,10.0,10.0,10.0,10.0,9.0,4.05,0
4,31080,https://www.airbnb.com/rooms/31080,2-story apartment + rooftop terrace,133488,Nienke,f,Zuid,Entire apartment,Entire home/apt,1 bath,...,2017-10-16,95,9.0,10.0,10.0,10.0,9.0,9.0,0.28,0


{0: 'Bos en Lommer',
 1: 'Buitenveldert - Zuidas',
 2: 'Centrum-Oost',
 3: 'Centrum-West',
 4: 'De Aker - Nieuw Sloten',
 5: 'De Baarsjes - Oud-West',
 6: 'De Pijp - Rivierenbuurt',
 7: 'Geuzenveld - Slotermeer',
 8: 'IJburg - Zeeburgereiland',
 9: 'Noord-Oost',
 10: 'Noord-West',
 11: 'Oostelijk Havengebied - Indische Buurt',
 12: 'Osdorp',
 13: 'Oud-Noord',
 14: 'Oud-Oost',
 15: 'Slotervaart',
 16: 'Watergraafsmeer',
 17: 'Westerpark',
 18: 'Zuid'}

In [7]:
neigh_df=pd.DataFrame(df["neighbourhood_cleansed"].value_counts()).sort_index().reset_index()
neigh_dict = neigh_df["index"].to_dict()
for value in neigh_dict.values():
    result = Neighbourhoods(neigh_name=value)
    session.add(result)
session.flush()


In [8]:


rt_df=pd.DataFrame(df["room_type"].value_counts()).sort_index().reset_index()
rt_dict = rt_df["index"].to_dict()
for value in rt_dict.values():
    result = Room_types(room_type_name=value)
    session.add(result)
session.flush()




In [9]:
property_type_df=pd.DataFrame(df["property_type"].value_counts()).sort_index().reset_index()
property_type_dict = property_type_df["index"].to_dict()
property_type_dict

for value in property_type_dict.values():
    result = Property_types(property_type_name=value)
    session.add(result)
session.flush()

In [10]:
session.commit()